# Necessary Imports

In [1]:
from arango import ArangoClient
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import folium
import networkx as nx

## Establishing Connection


In [5]:
ArangoClient = ArangoClient(hosts="http://127.0.0.1:8530")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

collections = db.collections()


## Preprocess Json data for easy loading

In [ ]:
def preprocess_json(input_file, output_file):
    with open(input_file, 'r') as f:
        # Read the entire file content
        data = f.read()

        # Split the content by newline character to handle multiple JSON objects
        json_objects = data.strip().split('\n')

    # Process each JSON object separately
    processed_data = []
    for json_str in json_objects:
        try:
            # Load each JSON object separately
            obj = json.loads(json_str)
            processed_data.append(obj)
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

    # Write processed data to the output file
    with open(output_file, 'w') as f:
        json.dump(processed_data, f)

if __name__ == "__main__":
    input_file_list = ["yelp_academic_dataset_business.json", "yelp_academic_dataset_checkin.json", "yelp_academic_dataset_tip.json"]
    output_file_list = ["yelp_academic_dataset_business_updated.json", "yelp_academic_dataset_checkin_updated.json", "yelp_academic_dataset_tip_updated.json"]
    for input_file, output_file in zip(input_file_list, output_file_list):
        preprocess_json(input_file, output_file)


In [ ]:

collection=["business","checkin","review","tip","user"]
file_list = ["yelp_academic_dataset_business_updated.json", "yelp_academic_dataset_checkin_updated.json", "yelp_academic_dataset_tip_updated.json"]
    
for collection_name, file in zip(collection, file_list):
    collection=db.collection(collection_name)
    json_file = file
    # Read the JSON data from file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Bulk insert the data into the collection
    collection.import_bulk(data)

- Since the size of the files of user and reviews are very high , we can use aranogosh(arangoimport) to load it entirely
- Command used: arangoimport --file "yelp_academic_dataset_review.json" --type json --collection "review" --server.endpoint "tcp://127.0.0.1:8530" --server.database "yelp_db" --server.username "super" --server.password "grantaccess" --create-collection true --threads 4 

## Defining Edge collection based on uploaded Collection

- There are only two edges in the collection : business_review and user_review

In [1]:
# Connect to ArangoDB
client = ArangoClient(hosts='http://127.0.0.1:8530')
db = client.db('yelp_db', username='root', password='')



In [18]:
# Function to insert records into an edge collection in batches
def insert_edges(collection_name, edge_documents, batch_size=1000):
    for i in range(0, len(edge_documents), batch_size):
        batch = edge_documents[i:i+batch_size]
        db[collection_name].import_bulk(batch)



In [24]:
def process_records_review_business(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"business/{record['business_id']}",  # Assuming '_key' is the document key
            "_to": f"review/{record['_key']}"
        })
    return edge_documents
def process_records_review_user(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from":f"user/{record['user_id']}",  # Assuming '_key' is the document key
            "_to": f"review/{record['_key']}"
        })
    return edge_documents

In [ ]:
reviews_collection = db.collection('review')

# Function to fetch reviews in batches
def fetch_reviews_in_batches(batch_size):
    reviews = []
    offset = 0

    while True:
        # Fetch a batch of reviews using a LIMIT and SKIP
        batch = reviews_collection.all(limit=batch_size, skip=offset)

        if not batch:
            break

        reviews.extend(batch)
        offset += batch_size
    return reviews

# Call the function with your desired batch size
batch_size = 50000  # Adjust as needed
reviewsrecords = fetch_reviews_in_batches(batch_size)

In [9]:
edge_documents_review_business = process_records_review_business(reviewsrecords)
insert_edges('business_review', edge_documents_review_business)

In [25]:
edge_documents_review_user = process_records_review_user(reviewsrecords)
insert_edges('user_review', edge_documents_review_user)

## Analysis on Data


In [2]:
ArangoClient = ArangoClient(hosts="http://192.168.1.120:8531")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

# Categories of Business and their count

In [3]:

categories=db.aql.execute("FOR b IN business RETURN b.categories")
categories_df = pd.DataFrame(categories, columns=['categories'])
print(categories_df.head())


                                          categories
0  Doctors, Traditional Chinese Medicine, Naturop...
1  Shipping Centers, Local Services, Notaries, Ma...
2  Department Stores, Shopping, Fashion, Home & G...
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                          Brewpubs, Breweries, Food


In [4]:


cat_list = []
for i in categories_df['categories']:
    if i:
        for j in i.split(","):
            cat_list.append(j.strip())
cat_df = pd.DataFrame(cat_list, columns=['categories'])
x = cat_df['categories'].value_counts()

print("There are ", len(x), " different types/categories of Businesses in Yelp!")

# Select the top 20 categories
x = x.sort_values(ascending=False).iloc[:20]

# Define colors for each bar
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
          '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff',
          '#00ffff', '#800000', '#008000', '#000080', '#808000']

# Create a Plotly bar chart
fig = go.Figure(data=[
    go.Bar(
        x=x.index,
        y=x.values,
        marker_color=colors
    )
])

fig.update_layout(
    title="What are the top categories?",
    xaxis=dict(title='Category'),
    yaxis=dict(title='# businesses'),
    xaxis_tickangle=-45,
    width=1000,
    height=500
)

fig.show()


There are  1311  different types/categories of Businesses in Yelp!


# Star rating of business

In [5]:
# fetch from buiness collection the star rating of each business
star=db.aql.execute("FOR b IN business RETURN b.stars")
star_df = pd.DataFrame(star, columns=['stars'])


# Assuming star_df is your DataFrame containing the 'stars' column
# Replace 'star_df' with the name of your DataFrame if different

# Step 1: Get the value counts of stars
star_counts = star_df['stars'].value_counts().sort_index()

# Step 2: Create the Plotly bar plot
data = [
    go.Bar(
        x=star_counts.index,
        y=star_counts.values,
        marker=dict(color='rgb(158,202,225)')
    )
]

layout = go.Layout(
    title='Distribution of Star Ratings',
    xaxis=dict(title='Stars'),
    yaxis=dict(title='Count'),
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
fig.show()


In [6]:
aql = """
    FOR r IN review
        LET date = DATE_ISO8601(DATE_TIMESTAMP(r.date))
        LET year = DATE_YEAR(date)
        COLLECT review_year = year
        WITH COUNT INTO review_count
        SORT review_year ASC
        RETURN { year: review_year, review_count: review_count }
"""

# Execute the query
cursor = db.aql.execute(aql)

# Prepare data for visualization
data = [(doc['year'], doc['review_count']) for doc in cursor]
years, counts = zip(*data)

# Convert data to DataFrame
df = pd.DataFrame({'Year': years, 'Review_Count': counts})

# Sort DataFrame by Year
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.sort_values(by='Year', inplace=True)

# Create Plotly figure
fig = go.Figure()

# Add bar trace
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Review_Count'],
    marker=dict(color=df['Year'].dt.year),
    text=df['Review_Count'],
    textposition='outside',
))

# Update layout
fig.update_layout(
    title='Number of Reviews per Year',
    xaxis_title='Year',
    yaxis_title='Number of Reviews',
    xaxis=dict(tickformat='%Y'),
    yaxis=dict(showgrid=True),
)

# Show plot
fig.show()


In [7]:
# Define your AQL query to retrieve latitude, longitude, and business name
aql_query = """
    FOR b IN business
    LIMIT 500
        RETURN { 
            name: b.name,
            latitude: b.latitude,
            longitude: b.longitude
        }
"""

# Execute the AQL query to retrieve business data
cursor = db.aql.execute(aql_query)

# Prepare data for visualization
data = [(doc['name'], doc['latitude'], doc['longitude']) for doc in cursor]
business_names, latitudes, longitudes = zip(*data)

# Create a DataFrame
df = pd.DataFrame({'Business': business_names, 'Latitude': latitudes, 'Longitude': longitudes})

# Create a base map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Add markers for each business location
for index, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Business']).add_to(m)

# Display the map
m


In [8]:
checkins=db.aql.execute("For c in checkin RETURN c")
checkins_df = pd.DataFrame(checkins, columns=['business','date'])
checkins_df['date'] = checkins_df['date'].apply(lambda x: pd.to_datetime(x.split(', ')))

In [9]:
dates = checkins_df['date'].explode()
# Sample data for the years 2019 and 2020
data_2019 = dates[dates.dt.year == 2019].sample(n=50000)
data_2020 = dates[dates.dt.year == 2020].sample(n=50000)



In [10]:
sampled_dates = pd.concat([data_2019, data_2020])

# Create histogram data
hist_data = go.Histogram(x=sampled_dates, marker_color='blue', opacity=0.7)

# Create layout
layout = go.Layout(title='Check-ins Distribution',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Count'),
                   bargap=0.05)

# Create figure
fig = go.Figure(data=[hist_data], layout=layout)

# Show the plot
fig.show()



In [11]:
# Define the AQL query
aql_query = """
LET businessKey = @businessKey
FOR v, e, p IN 1..3 OUTBOUND CONCAT('business/', @businessKey) business_review
RETURN v
"""

# Execute the AQL query
cursor = db.aql.execute(aql_query, bind_vars={'businessKey': 'tUFrWirKiKi_TAnsVWINQQ'})

# Initialize a directed graph
G = nx.DiGraph()

# Add nodes and edges to the graph
for document in cursor:
    review_id = document['_key']
    G.add_node(review_id)
    G.add_edge('business', review_id)

# Compute layout
pos = nx.spring_layout(G)

# Create a Plotly figure
fig = go.Figure()

# Add nodes to the figure
for node in G.nodes:
    fig.add_trace(go.Scatter(x=[pos[node][0]], y=[pos[node][1]], mode='markers', marker=dict(size=20, color='blue'), text=node))

# Add edges to the figure
for edge in G.edges:
    fig.add_trace(go.Scatter(x=[pos[edge[0]][0], pos[edge[1]][0]], y=[pos[edge[0]][1], pos[edge[1]][1]], mode='lines', line=dict(color='gray', width=1)))

# Update layout
fig.update_layout(title="Business-Review Visualization", showlegend=False, hovermode='closest', xaxis=dict(visible=False), yaxis=dict(visible=False))

# Show plot
fig.show()
